In [64]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from statsmodels import regression
import statsmodels.api as sm
import xlsxwriter

In [65]:
df = pd.read_excel("AQM_Excel_data.xlsx")
df

,Date,Country,CPI,GDP,Imports,Exports,Unemployment,House,Industrial_Index,GDP_Capita,ars,intr
0,2010-01-01,US,217.488,14721.350,2.408555e+03,-4.891790e+02,9.8,147.3940,91.6642,49903.0,345959.000000,0.1100
1,2010-02-01,US,217.281,14721.350,2.408555e+03,-4.891790e+02,9.8,145.6310,91.9940,49903.0,346895.000000,0.1300
2,2010-03-01,US,217.353,14721.350,2.408555e+03,-4.891790e+02,9.9,145.8590,92.5993,49903.0,354379.000000,0.1600
3,2010-04-01,US,217.403,14926.098,2.520175e+03,-5.191270e+02,9.9,146.3990,92.9436,50272.0,357365.000000,0.2000
4,2010-05-01,US,217.290,14926.098,2.520175e+03,-5.191270e+02,9.6,146.3890,94.2997,50272.0,354299.000000,0.2000
...,...,...,...,...,...,...,...,...,...,...,...,...
127,2020-08-01,FR,105.090,541368.400,5.236358e+10,9.323044e+12,9.3,4.9866,95.4100,NaN,5.944056,-0.1713
128,2020-09-01,FR,104.550,541368.400,5.227677e+10,9.323044e+12,9.3,4.9866,97.0800,NaN,2.907489,-0.2072
129,2020-10-01,FR,104.510,533547.700,5.278377e+10,9.297192e+12,8.5,NaN,98.9900,NaN,5.462555,-0.2966
130,2020-11-01,FR,104.730,533547.700,5.284258e+10,9.297192e+12,8.1,NaN,98.7300,NaN,-11.877729,-0.3262


In [71]:
df.dropna(inplace=True)
df.head()

,Date,Country,CPI,GDP,Imports,Exports,Unemployment,House,Industrial_Index,GDP_Capita,ars,intr
Date,,,,,,,,,,,,
2010-01-01,2010-01-01,US,217.488,14721.350,2408.555,-489.179,9.8,147.394,91.6642,49903.0,345959.0,0.11
2010-02-01,2010-02-01,US,217.281,14721.350,2408.555,-489.179,9.8,145.631,91.9940,49903.0,346895.0,0.13
2010-03-01,2010-03-01,US,217.353,14721.350,2408.555,-489.179,9.9,145.859,92.5993,49903.0,354379.0,0.16
2010-04-01,2010-04-01,US,217.403,14926.098,2520.175,-519.127,9.9,146.399,92.9436,50272.0,357365.0,0.20
2010-05-01,2010-05-01,US,217.290,14926.098,2520.175,-519.127,9.6,146.389,94.2997,50272.0,354299.0,0.20


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 2010-01-01 to 2019-12-01
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              120 non-null    object 
 1   Country           120 non-null    object 
 2   CPI               120 non-null    float64
 3   GDP               120 non-null    float64
 4   Imports           120 non-null    float64
 5   Exports           120 non-null    float64
 6   Unemployment      120 non-null    float64
 7   House             120 non-null    float64
 8   Industrial_Index  120 non-null    float64
 9   GDP_Capita        120 non-null    float64
 10  ars               120 non-null    float64
 11  intr              120 non-null    float64
dtypes: float64(10), object(2)
memory usage: 12.2+ KB


In [73]:
df['Date']=df['Date'].astype(str)
df

,Date,Country,CPI,GDP,Imports,Exports,Unemployment,House,Industrial_Index,GDP_Capita,ars,intr
Date,,,,,,,,,,,,
2010-01-01,2010-01-01,US,217.488,14721.350,2.408555e+03,-4.891790e+02,9.8,147.3940,91.664200,49903.000000,345959.0,0.1100
2010-02-01,2010-02-01,US,217.281,14721.350,2.408555e+03,-4.891790e+02,9.8,145.6310,91.994000,49903.000000,346895.0,0.1300
2010-03-01,2010-03-01,US,217.353,14721.350,2.408555e+03,-4.891790e+02,9.9,145.8590,92.599300,49903.000000,354379.0,0.1600
2010-04-01,2010-04-01,US,217.403,14926.098,2.520175e+03,-5.191270e+02,9.9,146.3990,92.943600,50272.000000,357365.0,0.2000
2010-05-01,2010-05-01,US,217.290,14926.098,2.520175e+03,-5.191270e+02,9.6,146.3890,94.299700,50272.000000,354299.0,0.2000
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,2019-08-01,IT,103.500,406401.700,3.943381e+10,4.369856e+10,9.6,0.0496,105.191234,35677.380539,100.5,1.4036
2019-09-01,2019-09-01,IT,102.900,406401.700,3.973357e+10,4.479502e+10,9.8,0.0496,104.891259,35677.380539,100.8,0.9021
2019-10-01,2019-10-01,IT,102.800,404610.000,3.876617e+10,4.601795e+10,9.7,-0.0892,103.691359,35677.380539,100.8,1.0036


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 2010-01-01 to 2019-12-01
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              120 non-null    object 
 1   Country           120 non-null    object 
 2   CPI               120 non-null    float64
 3   GDP               120 non-null    float64
 4   Imports           120 non-null    float64
 5   Exports           120 non-null    float64
 6   Unemployment      120 non-null    float64
 7   House             120 non-null    float64
 8   Industrial_Index  120 non-null    float64
 9   GDP_Capita        120 non-null    float64
 10  ars               120 non-null    float64
 11  intr              120 non-null    float64
dtypes: float64(10), object(2)
memory usage: 12.2+ KB


In [75]:
df = df.set_index(df['Date'])
df.head()

,Date,Country,CPI,GDP,Imports,Exports,Unemployment,House,Industrial_Index,GDP_Capita,ars,intr
Date,,,,,,,,,,,,
2010-01-01,2010-01-01,US,217.488,14721.350,2408.555,-489.179,9.8,147.394,91.6642,49903.0,345959.0,0.11
2010-02-01,2010-02-01,US,217.281,14721.350,2408.555,-489.179,9.8,145.631,91.9940,49903.0,346895.0,0.13
2010-03-01,2010-03-01,US,217.353,14721.350,2408.555,-489.179,9.9,145.859,92.5993,49903.0,354379.0,0.16
2010-04-01,2010-04-01,US,217.403,14926.098,2520.175,-519.127,9.9,146.399,92.9436,50272.0,357365.0,0.20
2010-05-01,2010-05-01,US,217.290,14926.098,2520.175,-519.127,9.6,146.389,94.2997,50272.0,354299.0,0.20


In [76]:
df = df.drop(axis = 1, columns = 'Date')
df.tail()

,Country,CPI,GDP,Imports,Exports,Unemployment,House,Industrial_Index,GDP_Capita,ars,intr
Date,,,,,,,,,,,
2019-08-01,IT,103.5,406401.7,3.943381e+10,4.369856e+10,9.6,0.0496,105.191234,35677.380539,100.5,1.4036
2019-09-01,IT,102.9,406401.7,3.973357e+10,4.479502e+10,9.8,0.0496,104.891259,35677.380539,100.8,0.9021
2019-10-01,IT,102.8,404610.0,3.876617e+10,4.601795e+10,9.7,-0.0892,103.691359,35677.380539,100.8,1.0036
2019-11-01,IT,102.6,404610.0,3.891874e+10,4.458105e+10,9.9,-0.0892,104.391301,35677.380539,100.1,1.2659
2019-12-01,IT,102.8,404610.0,3.864472e+10,4.505600e+10,9.9,-0.0892,101.491542,35677.380539,100.4,1.3700


In [20]:
for col in df.iteritems():
    print('plotting'+col[0])
    fig, ax = plt.subplots()
    indicator=df[col[0]]
    ax.plot(indicator, alpha=0.9, color='blue')
    plt.title(col[0])
    plt.savefig('Charts\Plot_'+col[0]+'.png')
    plt.close()

plottingCountry
plottingCPI
plottingGDP
plottingImports
plottingExports
plottingUnemployment
plottingHouse
plottingIndustrial_Index
plottingGDP_Capita
plottingars
plottingintr


In [82]:
import sqlite3
from sqlite3 import Error
 

""" create a database connection tool to a SQLite database """

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()
 
 
if __name__ == '__main__':
    create_connection(r"/Users/olympiagaki/Desktop/AQM_Project.db")

2.6.0


In [83]:
conn = sqlite3.connect(r"/Users/olympiagaki/Desktop/AQM_Project.db")

print("Opened database successfully")

Opened database successfully


In [85]:
# Create Table in using SQL and Python
conn = sqlite3.connect(r"/Users/olympiagaki/Desktop/AQM_Project.db")
c = conn.cursor()
conn.execute ('''CREATE TABLE Table5
        (DATE TIMESTAMP,
         COUNTRY           TEXT    NOT NULL,
         CPI            INT     NOT NULL,
         GDP            INT     NOT NULL,
         IMPORTS            INT     NOT NULL,
         EXPORTS            INT     NOT NULL,
         UNEMPLOYMENT            INT     NOT NULL,
         HOUSE            INT     NOT NULL,
         IND_INDEX            INT     NOT NULL,
         GDP_CAPITA            INT     NOT NULL,
         ARS            INT     NOT NULL,
         INT_RATE INT)''')

print("Table created successfully")


Table created successfully


In [86]:
df.to_sql('Table5', conn, if_exists='replace', index = True)
 
c.execute('''  
SELECT * FROM Table5
          ''')

for row in c.fetchall():
    print (row)

('US', 217.488, 14721.35, 2408.555, -489.179, 9.8, 147.394, 91.6642, 49903.0, 345959.0, 0.11)
('US', 217.281, 14721.35, 2408.555, -489.179, 9.8, 145.631, 91.994, 49903.0, 346895.0, 0.13)
('US', 217.353, 14721.35, 2408.555, -489.179, 9.9, 145.859, 92.5993, 49903.0, 354379.0, 0.16)
('US', 217.403, 14926.098, 2520.175, -519.127, 9.9, 146.399, 92.9436, 50272.0, 357365.0, 0.2)
('US', 217.29, 14926.098, 2520.175, -519.127, 9.6, 146.389, 94.2997, 50272.0, 354299.0, 0.2)
('US', 217.199, 14926.098, 2520.175, -519.127, 9.4, 145.716, 94.4397, 50272.0, 353763.0, 0.18)
('US', 217.605, 15079.917, 2612.581, -535.095, 9.4, 144.985, 94.8536, 50545.0, 354665.0, 0.18)
('US', 217.923, 15079.917, 2612.581, -535.095, 9.5, 143.908, 95.1448, 50545.0, 356810.0, 0.19)
('US', 218.275, 15079.917, 2612.581, -535.095, 9.5, 143.011, 95.3637, 50545.0, 359465.0, 0.19)
('US', 219.035, 15240.843, 2633.738, -512.21, 9.4, 142.522, 95.1109, 50698.0, 363340.0, 0.19)
('US', 219.59, 15240.843, 2633.738, -512.21, 9.8, 142.167,